In [89]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
%cd /content/drive/MyDrive/do_an

/content/drive/MyDrive/do_an


In [91]:
import pandas as pd

In [92]:
import numpy as np

In [93]:
import os

In [94]:
df_train = pd.read_csv('ml-1m/ratings.dat', sep = '::', names = ["user id", "item id", "rating", "timestamp"])

<ipython-input-94-f9098b30a6c7>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_train = pd.read_csv('ml-1m/ratings.dat', sep = '::', names = ["user id", "item id", "rating", "timestamp"])


In [95]:
df_user = pd.read_csv('ml-1m/users.dat', sep = '::', names = ['user', 'gender', 'age', 'occupation', 'zip_code'], encoding='latin-1')

<ipython-input-95-7ac5f8cb46c3>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_user = pd.read_csv('ml-1m/users.dat', sep = '::', names = ['user', 'gender', 'age', 'occupation', 'zip_code'], encoding='latin-1')


In [96]:
df_train = df_train[df_train['rating'] >= 3]

In [97]:
df_train['user id'].unique()

array([   1,    2,    3, ..., 6038, 6039, 6040])

In [98]:
df_train['item id'].max()

3952

In [99]:
df_train['user'] = df_train['user id'].copy()
df_train['item'] = df_train['item id'].copy()

<ipython-input-99-42ede7b853c7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['user'] = df_train['user id'].copy()
<ipython-input-99-42ede7b853c7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['item'] = df_train['item id'].copy()


In [100]:
path_save = 'datasets/ml_1m_one_task'

In [101]:
df_train_groupby_user = df_train.groupby('user', as_index = False).agg(lambda x : len(x))

In [102]:
#split = 0.95

In [103]:
split = 0.8

In [104]:
df_train_groupby_user['n_interactions'] = df_train_groupby_user['item'].copy()

In [105]:
list_users = df_train_groupby_user['user'][np.argsort(-df_train_groupby_user['item'].values)][:int(split * len(df_train_groupby_user))]

In [106]:
list_users

4167    4169
4275    4277
1679    1680
3616    3618
1014    1015
        ... 
1006    1007
2830    2831
2311    2312
5145    5147
3185    3186
Name: user, Length: 4831, dtype: int64

In [107]:
list_users_cold_start = df_train_groupby_user['user'][np.argsort(-df_train_groupby_user['item'].values)][int(split * len(df_train_groupby_user)):]

In [108]:
dict_user_to_id = {value : key + 1 for key, value in enumerate(list_users)}

In [109]:
max_user_id = max(list(dict_user_to_id.values()))

In [110]:
dict_user_to_id.update({value : key + 1 + max_user_id for key, value in enumerate(list_users_cold_start)})

In [111]:
list_users = np.arange(list_users.shape[0]) + 1

In [112]:
list_users_cold_start = np.arange(list_users_cold_start.shape[0]) + 1 + max_user_id

In [113]:
list_users_missing = [user for user in df_user['user'].unique() if user not in dict_user_to_id]
max_user_id = max(list(dict_user_to_id.values()))
dict_user_to_id.update({value : key + 1 + max_user_id for key, value in enumerate(list_users_missing)})

In [114]:
df_train

,user id,item id,rating,timestamp,user,item
0,1,1193,5,978300760,1,1193
1,1,661,3,978302109,1,661
2,1,914,3,978301968,1,914
3,1,3408,4,978300275,1,3408
4,1,2355,5,978824291,1,2355
...,...,...,...,...,...,...
1000203,6040,1090,3,956715518,6040,1090
1000205,6040,1094,5,956704887,6040,1094
1000206,6040,562,5,956704746,6040,562
1000207,6040,1096,4,956715648,6040,1096


In [115]:
df_train['user'] = df_train['user'].map(dict_user_to_id)

In [116]:
df_train['user_item'] = df_train['user'].astype(str) + "|" + df_train['item'].astype(str)

In [117]:
df_train

,user id,item id,rating,timestamp,user,item,user_item
0,1,1193,5,978300760,3883,1193,3883|1193
1,1,661,3,978302109,3883,661,3883|661
2,1,914,3,978301968,3883,914,3883|914
3,1,3408,4,978300275,3883,3408,3883|3408
4,1,2355,5,978824291,3883,2355,3883|2355
...,...,...,...,...,...,...,...
1000203,6040,1090,3,956715518,840,1090,840|1090
1000205,6040,1094,5,956704887,840,1094,840|1094
1000206,6040,562,5,956704746,840,562,840|562
1000207,6040,1096,4,956715648,840,1096,840|1096


In [118]:
split_cold_warm = 0.5
len_new_user = len(list_users_cold_start)

In [119]:
df_train_groupby_user['n_interactions'] = df_train_groupby_user['item'].copy()
#old
#df_train_groupby_user['n_interactions'][df_train_groupby_user['user'].isin(list_users) == False] = 0
df_train_groupby_user['n_interactions'][(df_train_groupby_user['user'].isin(list_users) == False) & (df_train_groupby_user['user'].isin(list_users_cold_start[:int(split_cold_warm * len_new_user)]) == False)] = 0

In [120]:
df_test_cold = df_train[df_train['user'].isin(list_users_cold_start[int(split_cold_warm * len_new_user):])]

In [121]:
df_test_cold

,user id,item id,rating,timestamp,user,item,user_item
233,4,3468,5,978294008,5805,3468,5805|3468
234,4,1210,3,978293924,5805,1210,5805|1210
235,4,2951,4,978294282,5805,2951,5805|2951
236,4,1214,4,978294260,5805,1214,5805|1214
237,4,1036,4,978294282,5805,1036,5805|1036
...,...,...,...,...,...,...,...
999738,6038,1296,5,956714684,5831,1296,5831|1296
999739,6038,1354,3,956714725,5831,1354,5831|1354
999742,6038,2716,3,956707604,5831,2716,5831|2716
999743,6038,3396,3,956706827,5831,3396,5831|3396


In [122]:
df_test_warm = df_train[df_train['user'].isin(list_users_cold_start[:int(split_cold_warm * len_new_user)])]

In [123]:
df_test_warm

,user id,item id,rating,timestamp,user,item,user_item
523,7,648,4,978234737,4893,648,4893|648
524,7,861,4,978234874,4893,861,4893|861
525,7,2916,5,978234842,4893,2916,4893|2916
526,7,3578,3,978234737,4893,3578,4893|3578
527,7,3793,3,978234737,4893,3793,4893|3793
...,...,...,...,...,...,...,...
997999,6029,2959,4,956722071,5062,2959,5062|2959
998000,6029,2628,3,956722153,5062,2628,5062|2628
998001,6029,2687,4,956722090,5062,2687,5062|2687
998002,6029,899,5,956721639,5062,899,5062|899


In [124]:
df_train_copy = df_train[df_train['user'].isin(list_users) | df_train['user'].isin(list_users_cold_start[:int(split_cold_warm * len_new_user)])]

In [125]:
df_test_warm = df_test_warm.sort_values(by = ['timestamp']).groupby('user', as_index = False).agg(lambda x : [elem for elem in x][-1])

In [126]:
df_train_copy = df_train_copy[df_train_copy['user_item'].isin(df_test_warm['user_item'].unique()) == False]

In [127]:
df_train = df_train[df_train['user'].isin(list_users) | df_train['user'].isin(list_users_cold_start[:int(split_cold_warm * len_new_user)])]

In [128]:
#split_val = 0.9

In [129]:
#df_val = df_train.sort_values(by = ['timestamp']).groupby('user', as_index = False).agg(lambda x : [elem for elem in x][int(split_val * len([elem for elem in x])):])

In [130]:
#df_val = df_val.set_index(['user']).apply(pd.Series.explode).reset_index()

In [131]:
#df_train = df_train.sort_values(by = ['timestamp']).groupby('user', as_index = False).agg(lambda x : [elem for elem in x][:int(split_val * len([elem for elem in x]))])

In [132]:
#df_train = df_train.set_index(['user']).apply(pd.Series.explode).reset_index()

In [133]:
#df_train = df_train[df_train['user_item'].isin(df_val['user_item'].unique()) == False]

In [134]:
df_train_groupby_user['n_interactions'] = df_train_groupby_user['item'].copy()
#old
#df_train_groupby_user['n_interactions'][df_train_groupby_user['user'].isin(list_users) == False] = 0
df_train_groupby_user['n_interactions'][(df_train_groupby_user['user'].isin(list_users) == False) & (df_train_groupby_user['user'].isin(list_users_cold_start[:int(split_cold_warm * len_new_user)]) == False)] = 0

In [135]:
df_val = df_train.sort_values(by = ['timestamp']).groupby('user', as_index = False).agg(lambda x : [elem for elem in x][-2])

In [136]:
df_train = df_train[df_train['user_item'].isin(df_val['user_item'].unique()) == False]

In [137]:
df_test = df_train.sort_values(by = ['timestamp']).groupby('user', as_index = False).agg(lambda x : [elem for elem in x][-1])

In [138]:
df_test = df_test[df_test['user'].isin(list_users)]

In [139]:
df_train = df_train[df_train['user_item'].isin(df_test['user_item'].unique()) == False]

In [140]:
def save_txt(data, filename):
    data_groupby_users = data.groupby("user", as_index = False).agg(lambda x : " ".join([str(elem) for elem in x]))
    data_groupby_users = data_groupby_users.sort_values(by = ["user"])
    final_data = data_groupby_users["user"].apply(lambda x : str(x) + "|") + data_groupby_users["item"]
    final_data = final_data.values
    final_data = "\n".join(final_data)
    f = open(filename, "w")
    f.write(final_data)
    f.close()
    print("Done")

In [141]:
save_txt(df_train, path_save + "/train_for_test.txt")

Done


In [142]:
df_train.groupby('user', as_index = False).agg(lambda x : len(x))['item'].min()

22

In [143]:
save_txt(df_test_cold, path_save + "/test_cold.txt")

Done


In [144]:
save_txt(df_test_warm, path_save + "/test_warm.txt")

Done


In [145]:
save_txt(df_train, path_save + "/train.txt")

Done


In [146]:
save_txt(df_val, path_save + "/val.txt")

Done


In [147]:
save_txt(df_test, path_save + "/test.txt")

Done


In [148]:
if False:
    #df_train_copy = df_train.copy()
    #data_copy = data_copy[data_copy["task_id"] == 0]
    #save_kg_txt = df_train_copy["user"].apply(lambda x : str(x) + " ") + df_train_copy['rating'].astype(str) + df_train_copy["item"].apply(lambda x : " " + str(x))
    save_kg_txt = df_train_copy["user"].apply(lambda x : str(x) + " ") + "0" + df_train_copy["item"].apply(lambda x : " " + str(x))
    save_kg_txt = "\n".join(save_kg_txt.values)

In [149]:
if True:
    #df_train_copy = df_train.copy()
    #data_copy = data_copy[data_copy["task_id"] == 0]
    #save_kg_txt = df_train_copy["user"].apply(lambda x : str(x) + " ") + df_train_copy['rating'].astype(str) + df_train_copy["item"].apply(lambda x : " " + str(x))
    #df_train_copy['rating'][df_train_copy['rating'] < 3] = 0
    #df_train_copy['rating'][df_train_copy['rating'] >= 3] = 1
    save_kg_txt = df_train_copy["user"].apply(lambda x : str(x) + " ") + df_train_copy['rating'].astype(int).astype(str) + df_train_copy["item"].apply(lambda x : " " + str(x))
    save_kg_txt = "\n".join(save_kg_txt.values)

In [150]:
df_train_copy

,user id,item id,rating,timestamp,user,item,user_item
0,1,1193,5,978300760,3883,1193,3883|1193
1,1,661,3,978302109,3883,661,3883|661
2,1,914,3,978301968,3883,914,3883|914
3,1,3408,4,978300275,3883,3408,3883|3408
4,1,2355,5,978824291,3883,2355,3883|2355
...,...,...,...,...,...,...,...
1000203,6040,1090,3,956715518,840,1090,840|1090
1000205,6040,1094,5,956704887,840,1094,840|1094
1000206,6040,562,5,956704746,840,562,840|562
1000207,6040,1096,4,956715648,840,1096,840|1096


In [151]:
save_kg_txt.split('\n')[0]

'3883 5 1193'

In [152]:
df_train['user'].nunique()

5435

In [153]:
f = open(path_save + "/kg_final.txt", "w")
f.write(save_kg_txt)
f.close()

In [154]:
categorical_user_columns = ['categories']

In [155]:
df_user['categories'] = df_user['gender'].astype(str) + ',' + df_user['occupation'].astype(str)

In [156]:
df_user['user'] = df_user['user'].map(dict_user_to_id)

In [157]:
df_train_groupby_user = df_train.groupby('user', as_index = False).agg(lambda x : len(x))

In [158]:
#old
#df_train_groupby_user['n_interactions'] = df_train['item'].copy()
df_train_groupby_user['n_interactions'] = df_train_groupby_user['item'].copy()

In [159]:
user_to_n_interactions = dict(df_train_groupby_user[['user', 'n_interactions']].values)

In [160]:
df_user['n_interactions'] = df_user['user'].map(user_to_n_interactions)
df_user['n_interactions'][pd.isnull(df_user['n_interactions'])] = 0

<ipython-input-160-d6cff776afa4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['n_interactions'][pd.isnull(df_user['n_interactions'])] = 0


In [161]:
numeric_user_columns = ['n_interactions', 'age']

In [162]:
with open(path_save + '/user_numeric_name_columns.txt', 'w') as f:
  f.write('\t'.join(numeric_user_columns))
  f.close()

In [163]:
with open(path_save + '/user_category_name_columns.txt', 'w') as f:
  f.write('\t'.join(categorical_user_columns))
  f.close()

In [164]:
with open(path_save + '/item_numeric_name_columns.txt', 'w') as f:
  f.write('\t'.join([]))
  f.close()

In [165]:
with open(path_save + '/item_category_name_columns.txt', 'w') as f:
  f.write('\t'.join([]))
  f.close()

In [166]:
def preprocessing_categories_id(data, column, begin_index, all_category_keys = None):
    if all_category_keys is None:
        all_category_ids = data[column][pd.isnull(data[column]) == False].apply(lambda x : str(x) + ",").sum()
        all_category_ids = str(all_category_ids)[:-1]
        all_category_ids = list(set(all_category_ids.split(",")))
        try:
            all_category_ids = np.sort(np.asarray(all_category_ids).astype(int))
        except:
            all_category_ids = np.asarray(all_category_ids)
        all_category_ids = all_category_ids.astype(str)
        new_category_ids = np.linspace(0, all_category_ids.shape[0] - 1, all_category_ids.shape[0]).astype(int).reshape(-1, 1) + begin_index
        all_category_keys = dict(np.hstack((all_category_ids.reshape(-1, 1), new_category_ids)))
        new_begin_index = np.max(new_category_ids[:, 0]) + 1
    else:
        new_begin_index = max(list(all_category_keys.values())) + 1
    data[column][pd.isnull(data[column]) == False] = data[column][pd.isnull(data[column]) == False].apply(lambda x : "\t".join([str(all_category_keys[elem]) for elem in str(x).split(",")])  + "\t")

    data[column][pd.isnull(data[column])] = "0"

    return all_category_keys, data, new_begin_index

In [167]:
begin_index = 0

In [168]:
for column in categorical_user_columns:
    if True:
        category_keys, df_user, begin_index = preprocessing_categories_id(df_user, column, begin_index)

<ipython-input-166-41a245489a5c>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column][pd.isnull(data[column]) == False] = data[column][pd.isnull(data[column]) == False].apply(lambda x : "\t".join([str(all_category_keys[elem]) for elem in str(x).split(",")])  + "\t")
<ipython-input-166-41a245489a5c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column][pd.isnull(data[column])] = "0"


In [169]:
df_user["all_categories"] = df_user[categorical_user_columns].sum(axis = 1)
df_user["all_categories"] = df_user["all_categories"].apply(lambda x : x[:-1])

In [170]:
df_user["all_categories"]

0         0\t4
1       13\t19
2       13\t14
3       13\t11
4        13\t8
         ...  
6035     0\t14
6036      0\t3
6037      0\t3
6038     0\t15
6039    13\t17
Name: all_categories, Length: 6040, dtype: object

In [171]:
df_user.to_csv(path_save + '/user_features.csv', index = False)

In [172]:
df_user = pd.read_csv('ml-1m/users.dat', sep = '::', names = ['user', 'gender', 'age', 'occupation', 'zipcode'], engine='python')

In [173]:
df_user['user'] = df_user['user'].map(dict_user_to_id)

In [174]:
df_item = pd.read_csv('ml-1m/movies_extrainfos.dat', sep = '::', names = ['item', 'title', 'year', 'rate', 'released', 'genre', 'director', 'writer', 'actor', 'plot', 'poster'], engine='python', encoding="utf-8")

In [175]:
def mapping(target_df, source_df, key):
    for column in source_df.columns:
        if key == column:
            continue
        target_df[column] = target_df[key].map(dict(source_df[[key, column]].values))
    return target_df

In [176]:
df_train = mapping(df_train, df_user, 'user')
df_train = mapping(df_train, df_item, 'item')

In [177]:
df_train

,user id,item id,rating,timestamp,user,item,user_item,gender,age,occupation,...,title,year,rate,released,genre,director,writer,actor,plot,poster
0,1,1193,5,978300760,3883,1193,3883|1193,F,1,10,...,One Flew Over the Cuckoo's Nest,1975.0,R,19 Nov 1975,Drama,Milos Forman,"Lawrence Hauben (screenplay), Bo Goldman (scre...","Michael Berryman, Peter Brocco, Dean R. Brooks...",McMurphy has a criminal past and has once agai...,https://m.media-amazon.com/images/M/MV5BZjA0OW...
1,1,661,3,978302109,3883,661,3883|661,F,1,10,...,James and the Giant Peach,1996.0,PG,12 Apr 1996,"Animation, Adventure, Family",Henry Selick,"Roald Dahl (based on the book by), Karey Kirkp...","Simon Callow, Richard Dreyfuss, Jane Leeves, J...",James' happy life at the English seaside is ru...,https://m.media-amazon.com/images/M/MV5BMTNkNW...
2,1,914,3,978301968,3883,914,3883|914,F,1,10,...,My Fair Lady,1964.0,G,25 Dec 1964,"Drama, Family, Musical",George Cukor,"Alan Jay Lerner (book), George Bernard Shaw (f...","Audrey Hepburn, Rex Harrison, Stanley Holloway...",Pompous phonetics professor Henry Higgins is s...,https://m.media-amazon.com/images/M/MV5BNGM0ZT...
3,1,3408,4,978300275,3883,3408,3883|3408,F,1,10,...,Erin Brockovich,2000.0,R,17 Mar 2000,"Biography, Drama",Steven Soderbergh,Susannah Grant,"Julia Roberts, David Brisbin, Dawn Didawick, A...",Erin Brockovich-Ellis is an unemployed single ...,https://m.media-amazon.com/images/M/MV5BYTA1NW...
4,1,2355,5,978824291,3883,2355,3883|2355,F,1,10,...,A Bug's Life,1998.0,G,25 Nov 1998,"Animation, Adventure, Comedy","John Lasseter, Andrew Stanton(co-director)","John Lasseter (original story by), Andrew Stan...","Dave Foley, Kevin Spacey, Julia Louis-Dreyfus,...","At an annual pace, a huge colony of ants is fo...",https://m.media-amazon.com/images/M/MV5BNThmZG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000203,6040,1090,3,956715518,840,1090,840|1090,M,25,6,...,Platoon,1986.0,R,06 Feb 1987,"Drama, War",Oliver Stone,Oliver Stone,"Tom Berenger, Keith David, Willem Dafoe, Fores...","Chris Taylor is a young, naive American who gi...",https://m.media-amazon.com/images/M/MV5BM2Y1NT...
1000205,6040,1094,5,956704887,840,1094,840|1094,M,25,6,...,The Crying Game,1992.0,R,19 Feb 1993,"Crime, Drama, Romance",Neil Jordan,Neil Jordan,"Forest Whitaker, Miranda Richardson, Stephen R...",An unlikely kind of friendship develops betwee...,https://m.media-amazon.com/images/M/MV5BYWU4Mj...
1000206,6040,562,5,956704746,840,562,840|562,M,25,6,...,Welcome to the Dollhouse,1995.0,R,24 May 1996,"Comedy, Drama",Todd Solondz,Todd Solondz,"Heather Matarazzo, Victoria Davis, Christina B...",Seventh-grade is no fun. Especially for Dawn W...,https://ia.media-imdb.com/images/M/MV5BOGE0N2U...
1000207,6040,1096,4,956715648,840,1096,840|1096,M,25,6,...,Sophie's Choice,1982.0,R,04 Mar 1983,"Drama, Romance",Alan J. Pakula,"William Styron (based on the novel: ""Sophie's ...","Meryl Streep, Kevin Kline, Peter MacNicol, Rit...",Sophie is the survivor of Nazi concentration c...,https://m.media-amazon.com/images/M/MV5BMTk3ND...


In [178]:
df_test_cold

,user id,item id,rating,timestamp,user,item,user_item
233,4,3468,5,978294008,5805,3468,5805|3468
234,4,1210,3,978293924,5805,1210,5805|1210
235,4,2951,4,978294282,5805,2951,5805|2951
236,4,1214,4,978294260,5805,1214,5805|1214
237,4,1036,4,978294282,5805,1036,5805|1036
...,...,...,...,...,...,...,...
999738,6038,1296,5,956714684,5831,1296,5831|1296
999739,6038,1354,3,956714725,5831,1354,5831|1354
999742,6038,2716,3,956707604,5831,2716,5831|2716
999743,6038,3396,3,956706827,5831,3396,5831|3396


In [179]:
df_test = pd.concat([df_test, df_test_warm])
df_test = mapping(df_test, df_user, 'user')
df_test = mapping(df_test, df_item, 'item')

In [180]:
df_test_cold

,user id,item id,rating,timestamp,user,item,user_item
233,4,3468,5,978294008,5805,3468,5805|3468
234,4,1210,3,978293924,5805,1210,5805|1210
235,4,2951,4,978294282,5805,2951,5805|2951
236,4,1214,4,978294260,5805,1214,5805|1214
237,4,1036,4,978294282,5805,1036,5805|1036
...,...,...,...,...,...,...,...
999738,6038,1296,5,956714684,5831,1296,5831|1296
999739,6038,1354,3,956714725,5831,1354,5831|1354
999742,6038,2716,3,956707604,5831,2716,5831|2716
999743,6038,3396,3,956706827,5831,3396,5831|3396


In [181]:
df_test_cold = mapping(df_test_cold, df_user, 'user')
df_test_cold = mapping(df_test_cold, df_item, 'item')

<ipython-input-175-a7bcc3da716e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_df[column] = target_df[key].map(dict(source_df[[key, column]].values))
<ipython-input-175-a7bcc3da716e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_df[column] = target_df[key].map(dict(source_df[[key, column]].values))
<ipython-input-175-a7bcc3da716e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [182]:
df_test_cold['zipcode']

233       02460
234       02460
235       02460
236       02460
237       02460
          ...  
999738    14706
999739    14706
999742    14706
999743    14706
999744    14706
Name: zipcode, Length: 11596, dtype: object

In [183]:
#df_test_cold['rate'] = df_test_cold['rating'].copy()
#df_test['rate'] = df_test['rating'].copy()
#df_train['rate'] = df_train['rating'].copy()
df_test_cold.to_csv(path_save + '/test_cold.csv', index = False)
df_test.to_csv(path_save + '/test.csv', index = False)
df_train.to_csv(path_save + '/train.csv', index = False)

In [184]:
df_user.to_csv(path_save + '/user.csv', index = False)
df_item.to_csv(path_save + '/item.csv', index = False)

In [185]:
#!python main_kgat_2.py --data_name ml_1m_one_task --use_pretrain 0 --cf_print_every 16 --kg_print_every 16 --evaluate_every 1 --Ks '[20, 50]' --cf_batch_size 1024  --lr 1e-4 --masking_train 1 --use_weight_embeds 1 --training_embed_weight 1 --use_train_data_to_train_weight_embed 1 --use_weight_embeds_no_grad 1

In [186]:
#!python main_kgat_2.py --data_name ml_1m_one_task --use_pretrain 0 --cf_print_every 16 --kg_print_every 16 --evaluate_every 1 --Ks '[20, 50]' --cf_batch_size 1024  --lr 1e-3 --masking_train 1 --use_weight_embeds 1 --training_embed_weight 1 --use_train_data_to_train_weight_embed 1 --full_sampling 1

In [187]:
#!python main_kgat_2.py --data_name ml_1m_one_task --use_pretrain 0 --cf_print_every 16 --kg_print_every 16 --evaluate_every 1 --Ks '[20, 50]' --cf_batch_size 1024  --lr 1e-2 --masking_train 1 --use_weight_embeds 1 --training_embed_weight 1 --use_train_data_to_train_weight_embed 0 --full_sampling 1

In [188]:
#!python main_kgat_2.py --data_name ml_1m_one_task --use_pretrain 0 --cf_print_every 16 --kg_print_every 16 --evaluate_every 1 --Ks '[20, 50]' --cf_batch_size 1024  --lr 1e-2 --masking_train 1 --use_weight_embeds 1 --training_embed_weight 0 --use_train_data_to_train_weight_embed 0 --full_sampling 1

In [189]:
#!python main_kgat_2.py --data_name ml_1m_one_task --use_pretrain 0 --cf_print_every 16 --kg_print_every 16 --evaluate_every 1 --Ks '[20, 50]' --cf_batch_size 1024  --lr 1e-2 --masking_train 1 --use_weight_embeds 0 --training_embed_weight 0 --use_train_data_to_train_weight_embed 0 --full_sampling 1

In [190]:
#!python main_kgat_raw.py --data_name ml_1m_one_task --use_pretrain 0 --cf_print_every 16 --kg_print_every 16 --evaluate_every 1 --Ks '[20, 50]' --cf_batch_size 1024  --lr 1e-2 --masking_train 1

In [191]:
#!python main_kgat_2_new_backward_2.py --data_name ml_1m_one_task --use_pretrain 0 --cf_print_every 16 --kg_print_every 16 --evaluate_every 1 --Ks '[20,50]' --cf_batch_size 1024  --lr 1e-3 --masking_train 1 --use_weight_embeds 1 --training_embed_weight 1 --use_train_data_to_train_weight_embed 0 --stopping_steps 20 --train_weight_embed_lr 1e-3 --use_binary_weight 0 --training_embed_weight_batch_size 200 --stop_training_embed_weight_epoch 1000